# Projeto Eletromag 21.2

## Ressonância magnética

- Andresa Bicudo
- Beatriz Cabral
- Vitor Bandeira

In [73]:
from math import *
from cmath import *
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg

## Etapa 1:

1) Elaboração do diagrama do modelo eletrodinâmico do circuito WPT

2) Código do modelo numérico

3) Construção de gráficos de I e I2 em função da frequência

4) Construção de gráficos das potências e da eficiência em função da frequência

### 1) Elaboração do diagrama do modelo eletrodinâmico do circuito WPT: 

Essa atividade consiste em especificar, por meio de um circuito, o modelo eletrodinâmico WPT usando abordagem fasorial. 

Para isso, o modelo eletrodinâmico do transformador deve ser adaptado de modo a considerar o parâmetro
𝑘 e os capacitores 𝐶1 e 𝐶2, lembrando que 𝑪𝟏 deve estarsérie com a bobina primária, e 𝑪𝟐 em paralelo
com a secundária; no projeto, vamos considerar que as bobinas transmissora e receptora sejam iguais,
ou seja, 𝐿1 = 𝐿2. 

Do mesmo modo, os capacitores também devem ser iguais (𝐶1 = 𝐶2), de modo que a
frequência de ressonância dos conjuntos 𝐿𝐶 seja a mesma no primário e secundário. 

Pesquise sobre como calcular a frequência de ressonância de um conjunto 𝐿𝐶.

Referências:

- https://www.youtube.com/watch?v=OwI0FXA-NIY

w = 1/sqrt(w*C)

In [74]:
![Circuito](./img/circ.pdf)

/bin/sh: -c: line 0: syntax error near unexpected token `./img/circ.pdf'
/bin/sh: -c: line 0: `[Circuito](./img/circ.pdf)'


### 2) Código do modelo numérico: 

Essa atividade consiste em escrever um código em Python que simule o modelo esquematizado no item anterior.

O código deve usar como parâmetros os valores das indutâncias, capacitâncias e resistências das bobinas primária e secundária (neste momento, usem valores quaisquer, mas razoáveis do ponto de vista físico). 

Recomenda-se a utilização, como ponto de partida, do código em Python usado no laboratório 2. 

Documente seu código explicando cada etapa do cálculo. 

Para as simulações iniciais, adotem valores razoáveis para indutâncias, frequência, resistências e capacitâncias a serem utilizados.

In [75]:
# Resistência da carga a ser utilizada
RC = 10 # definir

# Resistências internas das bobinas
R1 = 1 # definir
R2 = R1

# Capacitâncias (dos capacitores que vão gerar o efeito de ressonância)
C1 = 1e-6 # definir
C2 = C1

# Indutâncias (dos indutores que fazem parte da bobina)
L1 = 1e-6 # definir
L2 = L1

# Indutância mútua
M = math.sqrt(L1*L2)

print(f"A capacitância no primário C1 é de {C1} F (e está em série)")
print(f"A capacitância no secundário C2 é de {C2} F (e está em paralelo)\n")

print(f"A indutância no primário L1 é de {L1} H (e está em série)")
print(f"A indutância no secundário L2 é de {L2} H (e está em paralelo)")

A capacitância no primário C1 é de 1e-06 F (e está em série)
A capacitância no secundário C2 é de 1e-06 F (e está em paralelo)

A indutância no primário L1 é de 1e-06 H (e está em série)
A indutância no secundário L2 é de 1e-06 H (e está em paralelo)


In [76]:
freq = 440 # definir
w = 2*math.pi*freq
Uf = 1 # definir
k = 0.7 # chutar um valor - 0 a 1
M = k*math.sqrt(L1*L2)

# a freq de ressonância em LC tem de que ser igual nos dois conjuntos
w_lc1 = math.sqrt(1/L1*C1)
w_lc2 = w_lc1 # são iguais porque os parâmetros do primário e secundário são iguais

# Reatâncias capacitivas
XC1 = 1/(1j*w*C1)
XC2 #= XC1

# Reatâncias indutivas
XL1 = 1j*w*L1
XL2 = XL1

# Reatância indutiva (da mútua)
XM = 1j*w*M

print(f"A reatância capacitiva no primário XC1 é de {XC1}")
print(f"A reatância capacitiva no secundário XC2 é de {XC2}\n")

print(f"A reatância indutiva no primário XL1 é de {XL1}")
print(f"A reatância indutiva no secundário XL2 é de {XL2}")

A reatância capacitiva no primário XC1 é de -361.71577975430756j
A reatância capacitiva no secundário XC2 é de -0.3617157797543076j

A reatância indutiva no primário XL1 é de 0.002764601535159018j
A reatância indutiva no secundário XL2 é de 0.002764601535159018j


In [77]:
# Reatância total do circuito primário
X1 = XL1 + XC1 + R1 # série

# Reatância total do circuito secundário
X2 = XL2 + R2 + (XC2+R2)/(XC2*R2) # paralelo

print(f"A reatância total no primário X1 é de {X1}")
print(f"A reatância total no secundário X2 é de {X2}\n")

'''
As mudanças necessárias na matriz do cálculo das correntes do transformador são
incluir os capacitores (em série no primário e em paralelo no secundário)

No primário, somamos a reatância capacitiva aos outros valores de 
'''

def CalcularTransformadorNovo(Uf, RC):
    # calculando a reatância equivalente do primário (tudo em série)
    X1 = R1+XL1+XC1

    # calculando a reatância equivalente do paralelo
    # entre RC e o capacitor 2
    XRC = (RC*XC2)/(RC+XC2)

    # somando isso à reatância indutiva e R2 (que representam a bobina)
    # e que estão em série com o resto
    X2 = XL2+R2+XRC

    # criando a matriz da impedância
    # (que envolve os fatores que multiplicam as correntes I1 e I2)
    Z=np.array([[X1, -XM],[-XM, X2]])

    # criando a matriz das tensões
    V=np.array([Uf,0])

    # calculando a multiplicação das duas matrizes anteriores, 
    # que resultam nas correntes que queremos encontrar
    i=np.dot(np.linalg.inv(Z),V)

    return i[0], i[1]

# Calculando as correntes através das matrizes
I1, I2 = CalcularTransformadorNovo(Uf, RC)
print(f"A corrente no primário I1 é de:\n{I1}")
print(f"A corrente no secundário I2 é de:\n{I2}")

A reatância total no primário X1 é de (1-361.7130151527724j)
A reatância total no secundário X2 é de (2+2.7673661366941773j)

A corrente no primário I1 é de:
(7.643105223306782e-06+0.0027646015437440418j)
A corrente no secundário I2 é de:
(-4.698020436942963e-06-1.6478167956581016e-06j)


### 3) Construção de gráficos de I1 e I2 em função da frequência: 

Essa atividade consiste em fazer um gráfico dos módulos das correntes I1 e I2 em função 
da frequência da fonte, para um 𝑘 (coeficiente de acoplamento) qualquer inicialmente fixo.

A frequência para a qual as correntes são máximas é coincidente com a frequência de ressonância 
que vocês obtiveram na atividade 1? 

O que acontece com os gráficos dos módulos das correntes 
quando se altera o 𝑘 do circuito? 

Documente os resultados que obtiveram, pensando em gráficos 
adicionais que possam esclarecer o funcionamento do circuito.

In [ ]:
# V2 - precisa adaptar porque agora tem o paralelo

### 4) Construção de gráficos das potências e da eficiência em função da frequência: 

Por fim, a última atividade da Etapa 1 consiste em construir gráficos das potências reais de entrada e de saída em função da frequência. 

Além desses gráficos, vocês construirão um terceiro gráfico que relacionará a eficiência (ou o ganho de potência) do seu sistema com frequência da fonte. Documente os resultados que obtiveram, pensando em gráficos adicionais que possam esclarecer o funcionamento do circuito.

## Etapa 2 – Presencial - Atividades

1) Montagem do circuito de alimentação do primário

2) Projeto das bobinas de transmissão e recepção

3) Montagem das bobinas primária e secundária

4) Atualização da simulação da Etapa 1

5) Montagem do circuito WPT e medição das distâncias e potências transmitidas

6) Preparação da apresentação do projeto, a ser realizada no dia 29/11

### 1) Montagem do circuito de alimentação do primário: 

Pelo fato de o gerador de sinais não ter potência suficiente para alimentar a bobina primária do WPT, vamos precisar trabalhar com um circuito denominado “meia ponte” (ou half bridge), capaz de isolar a excitação na frequência de ressonância (circuito de controle), proveniente do gerador de sinais, da alimentação da bobina primária (circuito de potência). 

Observe o circuito que vamos montar na figura a seguir. 

Na entrada do circuito (pino 2 do IR2302) conectaremos o gerador de sinais ou o Arduíno, que deve fornecer um sinal de onda quadrada na frequência de ressonância (entre 0 𝑉 e 𝑉𝑐𝑐). 

O circuito integrado IR2302 controla o acionamento das chaves eletrônicas (MOSFETs), acionando-as alternadamente nos semiciclos nulo e positivo do sinal quadrado. Na montagem do circuito, é fundamental verificar o sinal de onda quadrada nos pontos de entrada e saída com o auxílio de um osciloscópio (esse sinal deve ir de 0 𝑉 a 𝑉𝑐𝑐, sendo
𝑉𝑐𝑐 máximo igual a 5 𝑉). Perguntem aos professores e aos técnicos do LFI sobre o funcionamento desse circuito, não deixando de fazer anotações sobre ele, principalmente compreendo por que não é possível conectar o Arduíno ou o gerador de sinais diretamente à bobina primária.

IMPORTANTE: todos os componentes para a montagem desse circuito encontram-se no LFI.

### 2) Projeto das bobinas de transmissão e recepção: 

Seu grupo deve projetar as bobinas que serão utilizadas no circuito. 

Vocês devem decidir qual o formato, raio/largura, bitola do fio e número de voltas para certa indutância pretendida, além de definirem a frequência em que pretendem transmitir energia pelo ar (devido aos limites do circuito meia ponte, vamos limitar a frequência a um máximo de 50𝑘𝐻𝑧). 

Há inúmeros sites/artigos que fornecem equações ou permitem o cálculo aproximado da indutância de uma bobina com formato específico e com núcleo de ar. 

Os professores e os técnicos podem ajudar a escolher sites confiáveis ou equações já testadas em laboratório.

Definidas a indutância e a frequência de operação, é possível calcular a capacitância dos capacitores
𝐶1 e 𝐶2 de modo a que o sistema trabalhe na frequência de ressonância pretendida.

### 3) Montagem das bobinas primária e secundária: 

Após estimar os valores de indutância usando equações e/ou softwares, assim como os valores de frequência e dos capacitores, vocês devem construir as bobinas e medir suas indutâncias e resistências usando o medidor LCR de alta precisão que temos no laboratório (certifiquem-se que estão medindo as indutâncias e as resistências das bobinas na
frequência de operação do circuito de vocês). 

Será necessário adaptar o projeto aos componentes
existentes, o que os pode levar a refazer/ajustar as bobinas. Quando maior a frequência, maiores os
valores das resistências 𝑅1 e 𝑅2 das bobinas devido ao “efeito pelicular” (ou skin effect). 

Pesquisem sobre esse efeito.

### 4) Atualização da simulação da Etapa 1: 

Como agora vocês têm os parâmetros medidos das bobinas e
capacitores, devem atualizar os parâmetros do modelo construído na Etapa 1. 

Como estamos trabalhando com uma tensão quadrada na entrada da bobina primária, precisamos fazer uma pequena
alteração na amplitude da função senoidal usada na simulação. 

Para isso, utilize a expressão:

𝑽𝟏(𝒕) = 𝟐𝝅𝑽𝒑 𝒄𝒐𝒔(𝝎𝒕)

Nessa expressão, 𝑉𝑝 é a amplitude da onda quadrada (alimentação dos MOSFETs). 

Os professores ficarão felizes em explicar por que isso é necessário.

### 5) Montagem do circuito WPT e medição das distâncias e potências transmitidas: 

Nessa atividade, vocês conectarão as bobinas ao circuito e iniciarão o experimento, filmando as situações experimentadas para edição final de um vídeo demonstração. Comparem a medição da tensão 𝑉2( ) àquela simulada pelo modelo. 

Percebam que é possível ajustar o coeficiente de acoplamento de tal modo a calibrar o modelo frente às medições. 

É possível construir um circuito retificador simples, com quatro diodos e um capacitor eletrolítico, a ser conectado à saída do circuito secundário - tensão 𝑉2( ) -, de modo que
uma carga DC possa ser conectada ao receptor WPT.

### 6) Preparação da apresentação do projeto, a ser realizada no dia 29/11: 

Em 5 ou 6 slides, vocês devem explicar o circuito, o processo de construção das bobinas, as medições e validações realizadas. 

É muito
importante que elaborem uma tabela com as medições das potências transmitidas e a que distâncias
ocorreram as transmissões. Observem a tabela a seguir para orientação, a qual deve ser preenchida
com um mínimo de 5 medições.

In [78]:
# RASCUNHO
def CalcularTransformador(Uf, RC):
    Z=np.array([[R1+XL1, -XM],[-XM, XL2+R2+RC]])
    V=np.array([Uf,0])
    i=np.dot(np.linalg.inv(Z),V)
    return i[0], i[1]